In [1]:
import os
import sys
import shutil
import time
import pytz
import traceback

import Ska.engarchive.fetch as fetch
import Chandra.Time

import datetime as dt
import matplotlib.dates as mdate
from matplotlib import gridspec

import matplotlib.pyplot as plt
import mpld3

import numpy as np
import pandas as pd
from astropy.table import Table

from Scripts.msidlists import *
from Scripts.event_times import *
from Scripts.plot_stylers import *

from tqdm import tqdm as progressbar

from hrcsentinel import hrccore as hrc


# allow_subset=True should let us draw more data points
# fetch.data_source.set('maude allow_subset=True')
fetch.data_source.set('cxc', 'maude allow_subset=True')
# fetch.data_source.set('maude')

hrc.styleplots()
labelsizes = 15
plt.rcParams['axes.titlesize'] = labelsizes + 2
plt.rcParams['axes.labelsize'] = labelsizes
plt.rcParams['xtick.labelsize'] = labelsizes
plt.rcParams['ytick.labelsize'] = labelsizes

%config InlineBackend.figure_format = 'retina'

In [2]:
davec_temperature_msids

['2PMT1T',
 '2PMT2T',
 '2DTSTATT',
 '2DCENTRT',
 '2FHTRMZT',
 '2CHTRPZT',
 '2FRADPYT',
 '2CEAHVPT',
 '2CONDMXT',
 '2UVLSPXT']

In [3]:
fetch.data_source.set('cxc') # just use ska for this

In [4]:
def convert_to_doy(datetime_start):
    '''
    Return a string like '2020:237' that will be passed to start= in fetch.get_telem()
    '''

    year = datetime_start.year
    day_of_year = datetime_start.timetuple().tm_yday
    doystring = '{}:{}'.format(year, day_of_year)

    return doystring

In [5]:
data_start = convert_to_doy(oneday_pre_anomaly)
data_start

'2020:236'

lets go one 4 days post anomaly:

In [6]:
data_stop = '2020:241'

Fetch the data

In [7]:
dat = fetch.get_telem(davec_temperature_msids, start=data_start, stop=data_stop, sampling='5min')

Fetching 5min-resolution data for MSIDS=['2PMT1T', '2PMT2T', '2DTSTATT', '2DCENTRT', '2FHTRMZT', '2CHTRPZT', '2FRADPYT', '2CEAHVPT', '2CONDMXT', '2UVLSPXT']
  from 2020:236:12:00:00.000 to 2020:241:12:00:00.000


In [8]:
dat

MSIDset([('2PMT1T',
          <MSID start=2020:236:12:00:00.000 stop=2020:241:12:00:00.000 len=1317 dtype=float32 unit=DEGC stat=5min>),
         ('2PMT2T',
          <MSID start=2020:236:12:00:00.000 stop=2020:241:12:00:00.000 len=1317 dtype=float32 unit=DEGC stat=5min>),
         ('2DTSTATT',
          <MSID start=2020:236:12:00:00.000 stop=2020:241:12:00:00.000 len=1317 dtype=float32 unit=DEGC stat=5min>),
         ('2DCENTRT',
          <MSID start=2020:236:12:00:00.000 stop=2020:241:12:00:00.000 len=1317 dtype=float32 unit=DEGC stat=5min>),
         ('2FHTRMZT',
          <MSID start=2020:236:12:00:00.000 stop=2020:241:12:00:00.000 len=1317 dtype=float32 unit=DEGC stat=5min>),
         ('2CHTRPZT',
          <MSID start=2020:236:12:00:00.000 stop=2020:241:12:00:00.000 len=1317 dtype=float32 unit=DEGC stat=5min>),
         ('2FRADPYT',
          <MSID start=2020:236:12:00:00.000 stop=2020:241:12:00:00.000 len=1317 dtype=float32 unit=DEGC stat=5min>),
         ('2CEAHVPT',
         

In [9]:
def make_excel_datestring(date):
    date.replace(tzinfo=None)
    print(type(date))
    epoch = dt.datetime(1899, 12,30) # Excel epoch is 1900-01-00
    epoch.replace(tzinfo=None)
    delta = date - epoch
    return float(delta.days) + (float(delta.seconds) / 86400)

In [10]:
columns = [dat["2PMT1T"].times, 
           mdate.num2date(hrc.convert_chandra_time(dat["2PMT1T"].times)),        
          dat["2PMT1T"].vals, 
          dat["2PMT2T"].vals,
          dat["2DTSTATT"].vals,
          dat["2DCENTRT"].vals,
          dat["2FHTRMZT"].vals,
          dat["2CHTRPZT"].vals,
          dat["2FRADPYT"].vals,
          dat["2CEAHVPT"].vals,
          dat["2CONDMXT"].vals,
          dat["2UVLSPXT"].vals]

column_names = ["Chandra Time (seconds since 1998 Jan 1, 00:00:00 TT, or MJD 50814.0 TT)",
                "Human-readable Time",
               "PMT1 EED Temperature 2PMT1T (C)",
               "PMT2 EED Temperature 2PMT2T (C)",
               "OutDet1 Temperature 2DTSTATT (C)",
               "OutDet1 Temperature 2DCENTRT (C)",
               "FEA Box Temperature 2FHTRMZT (C)",
               "FEA Box Temperature 2CHTRPZT (C) (C)",
               "+Y EED Temperature 2FRADPYT (C)",
               "-Y EED Temperature 2CEAHVPT (C)",
               "Conduit Temperature 2CONDMXT (C)",
                "Snout Temperature 2UVLSPXT (C)"]

In [11]:
a = pd.DataFrame(data=columns).T
a.columns=column_names
a

,"Chandra Time (seconds since 1998 Jan 1, 00:00:00 TT, or MJD 50814.0 TT)",Human-readable Time,PMT1 EED Temperature 2PMT1T (C),PMT2 EED Temperature 2PMT2T (C),OutDet1 Temperature 2DTSTATT (C),OutDet1 Temperature 2DCENTRT (C),FEA Box Temperature 2FHTRMZT (C),FEA Box Temperature 2CHTRPZT (C) (C),+Y EED Temperature 2FRADPYT (C),-Y EED Temperature 2CEAHVPT (C),Conduit Temperature 2CONDMXT (C),Snout Temperature 2UVLSPXT (C)
0,7.14571e+08,2020-08-23 12:04:12+00:00,20.7229,24.0356,19.6715,18.6448,22.3485,22.9038,18.1404,26.3868,20.7229,12.9421
1,7.14572e+08,2020-08-23 12:09:40+00:00,20.7229,24.0071,19.6715,18.6448,22.3485,22.9038,18.1404,26.3868,20.7229,12.9421
2,7.14572e+08,2020-08-23 12:15:08+00:00,20.7229,24.0356,19.6715,18.6448,22.3485,22.9038,18.1404,26.3868,20.7229,12.9421
3,7.14572e+08,2020-08-23 12:20:36+00:00,20.7229,24.0071,19.6715,18.6448,22.3485,22.9038,18.1404,26.3868,20.7229,12.9421
4,7.14573e+08,2020-08-23 12:26:04+00:00,20.7229,24.0071,19.6715,18.6448,22.3485,22.9038,18.1404,26.3868,20.7229,12.9421
5,7.14573e+08,2020-08-23 12:31:32+00:00,20.7229,24.0356,19.6715,18.6448,22.3485,22.9038,18.1404,26.3868,20.7229,12.9421
6,7.14573e+08,2020-08-23 12:37:00+00:00,20.7229,24.0071,19.6715,18.6448,22.3485,22.9038,18.1404,26.3868,20.7229,12.9421
7,7.14574e+08,2020-08-23 12:42:28+00:00,20.7229,24.0356,19.6715,18.3674,22.3485,22.9038,18.1404,26.3868,20.7229,12.9421
8,7.14574e+08,2020-08-23 12:47:56+00:00,20.7229,24.0356,19.6715,18.1404,22.184,22.9038,17.891,26.3868,20.7229,12.9421
9,7.14574e+08,2020-08-23 12:53:24+00:00,20.7229,24.0356,19.6715,18.1404,21.8,22.9038,17.6417,26.3868,20.7229,12.9421


In [12]:
a.to_excel('temperatures_for_davec.xlsx')

In [13]:
eventdate

[datetime.datetime(2020, 8, 24, 3, 45, 54, 967615, tzinfo=<matplotlib.dates._UTC object at 0x7fb5c8d90518>)]